# 第8章:ニューラルネット

## 70.単語ベクトルの和による特徴量
問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例xi
の特徴ベクトルxi
を並べた行列X
と，正解ラベルを並べた行列（ベクトル）Y
を作成したい．

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読込
df = pd.read_csv('./NewsAggregatorDataset/newsCorpora.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# データの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

# データの分割
train, valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=valid_test['CATEGORY'])

# データの保存
train.to_csv('./train.txt', sep='\t', index=False)
valid.to_csv('./valid.txt', sep='\t', index=False)
test.to_csv('./test.txt', sep='\t', index=False)

# 事例数の確認
print('【学習データ】')
print(train['CATEGORY'].value_counts())
print('【検証データ】')
print(valid['CATEGORY'].value_counts())
print('【評価データ】')
print(test['CATEGORY'].value_counts())

【学習データ】
b    4502
e    4223
t    1219
m     728
Name: CATEGORY, dtype: int64
【検証データ】
b    562
e    528
t    153
m     91
Name: CATEGORY, dtype: int64
【評価データ】
b    563
e    528
t    152
m     91
Name: CATEGORY, dtype: int64


In [10]:
import gdown
from gensim.models import KeyedVectors

# ダウンロードファイルのロード  #単語ベクトルを読み込む
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [11]:
import string
import torch

def transform_w2v(text):    #string.punctuation='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    words = text.translate(table).split()  # string.punctuationをスペースに置換後、スペースで分割してリスト化
    vec = [model[word] for word in words if word in model]  # 1語ずつベクトル化

    return torch.tensor(sum(vec) / len(vec))  # 平均ベクトルをTensor型に変換して出力

In [12]:
# 特徴ベクトルの作成
X_train = torch.stack([transform_w2v(text) for text in train['TITLE']])
X_valid = torch.stack([transform_w2v(text) for text in valid['TITLE']])
X_test = torch.stack([transform_w2v(text) for text in test['TITLE']])

print(X_train.size())
print(X_train)

torch.Size([10672, 300])
tensor([[-0.0170,  0.1318, -0.0728,  ...,  0.0395,  0.0223,  0.0184],
        [-0.1119, -0.0523, -0.1002,  ...,  0.0319, -0.0237, -0.0425],
        [-0.0636, -0.0228, -0.0005,  ..., -0.0280,  0.1057,  0.0396],
        ...,
        [ 0.0301, -0.0355, -0.0082,  ..., -0.0045,  0.0606, -0.0281],
        [ 0.0002,  0.0442, -0.0424,  ..., -0.0507,  0.0283,  0.0365],
        [ 0.0242,  0.0418,  0.1211,  ..., -0.0564,  0.0144,  0.0277]])


In [13]:
# ラベルベクトルの作成
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
y_train = torch.tensor(train['CATEGORY'].map(lambda x: category_dict[x]).values)
y_valid = torch.tensor(valid['CATEGORY'].map(lambda x: category_dict[x]).values)
y_test = torch.tensor(test['CATEGORY'].map(lambda x: category_dict[x]).values)

print(y_train.size())
print(y_train)

torch.Size([10672])
tensor([0, 0, 0,  ..., 0, 2, 2])


In [14]:
# 保存
torch.save(X_train, 'X_train.pt')
torch.save(X_valid, 'X_valid.pt')
torch.save(X_test, 'X_test.pt')
torch.save(y_train, 'y_train.pt')
torch.save(y_valid, 'y_valid.pt')
torch.save(y_test, 'y_test.pt')

## 71.単層ニューラルネットワークによる予測
問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

In [15]:
from torch import nn

class SLPNet(nn.Module): #単層ニューラルネットワークを定義
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc = nn.Linear(input_size, output_size, bias=False)   #ネットワーク層
        nn.init.normal_(self.fc.weight, 0.0, 1.0)  # 正規乱数で重みを初期化

    def forward(self, x):   #順伝播で通るレイヤーを配置
        x = self.fc(x)
        return x

In [36]:
model = SLPNet(300, 4)  # 単層ニューラルネットワークの初期化   #定義したモデルを初期化
y_hat_1 = torch.softmax(model(X_train[:1]), dim=-1)      #題の計算
print(y_hat_1.size())
print(y_hat_1)

torch.Size([1, 4])
tensor([[0.5499, 0.4140, 0.0216, 0.0145]], grad_fn=<SoftmaxBackward0>)


In [29]:
Y_hat = torch.softmax(model.forward(X_train[:4]), dim=-1)   #dim=-1 最後の次元の要素の和が1になる
print(Y_hat.size())
print(Y_hat)

torch.Size([4, 4])
tensor([[0.1147, 0.8330, 0.0075, 0.0449],
        [0.5347, 0.4454, 0.0031, 0.0168],
        [0.4620, 0.1664, 0.0440, 0.3275],
        [0.1963, 0.5527, 0.0687, 0.1822]], grad_fn=<SoftmaxBackward0>)


## 72.損失と勾配の計算

In [18]:
criterion = nn.CrossEntropyLoss()

In [40]:
l_1 = criterion(model(X_train[:1]), y_train[:1])  # 入力ベクトルはsoftmax前の値
model.zero_grad()  # 勾配をゼロで初期化
l_1.backward()  # 勾配を計算
print(f'損失: {l_1:.4f}')
print(f'勾配:\n{model.fc.weight.grad}')
print("-------------------------------")
print(model(X_train[:1]))
print(y_train[:1])

損失: 0.5981
勾配:
tensor([[ 0.0076, -0.0593,  0.0327,  ..., -0.0178, -0.0100, -0.0083],
        [-0.0070,  0.0546, -0.0301,  ...,  0.0164,  0.0092,  0.0076],
        [-0.0004,  0.0028, -0.0016,  ...,  0.0009,  0.0005,  0.0004],
        [-0.0002,  0.0019, -0.0011,  ...,  0.0006,  0.0003,  0.0003]])
-------------------------------
tensor([[ 1.8965,  1.6128, -1.3407, -1.7408]], grad_fn=<MmBackward0>)
tensor([0])


In [20]:
l = criterion(model(X_train[:4]), y_train[:4])
model.zero_grad()
l.backward()
print(f'損失: {l:.4f}')
print(f'勾配:\n{model.fc.weight.grad}')

損失: 1.6214
勾配:
tensor([[ 0.0179, -0.0496,  0.0236,  ..., -0.0096, -0.0107, -0.0061],
        [-0.0028,  0.0080, -0.0007,  ..., -0.0002,  0.0016,  0.0011],
        [-0.0096,  0.0319, -0.0192,  ...,  0.0091,  0.0060,  0.0036],
        [-0.0054,  0.0097, -0.0037,  ...,  0.0008,  0.0031,  0.0014]])


## 73.確率的勾配降下法による学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列W
を学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

In [21]:
from torch.utils.data import Dataset

# 特徴ベクトルとラベルベクトルを合わせて保持することができる型
class NewsDataset(Dataset):
    def __init__(self, X, y):  # datasetの構成要素を指定
        self.X = X
        self.y = y

    def __len__(self):  # len(dataset)で返す値を指定
        return len(self.y)

    def __getitem__(self, idx):  # dataset[idx]で返す値を指定
        return [self.X[idx], self.y[idx]]

In [23]:
from torch.utils.data import DataLoader

# Datasetの作成
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)
dataset_test = NewsDataset(X_test, y_test)

# Dataloaderの作成
#batch_sizeのデータを取り出す
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=False)

In [41]:
# モデルの定義
model = SLPNet(300, 4)

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# オプティマイザの定義 #最適化 #勾配計算
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)    #確率的勾配降下法

# 学習
num_epochs = 10
for epoch in range(num_epochs):
    # 訓練モードに設定
    model.train()
    loss_train = 0.0
    for i, (inputs, labels) in enumerate(dataloader_train):
        # 勾配をゼロで初期化
        optimizer.zero_grad()

        # 順伝播 + 誤差逆伝播 + 重み更新
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 損失を記録
        loss_train += loss.item()
        print

    # バッチ単位の平均損失計算   #1epochごと?
    loss_train = loss_train / i
    #print(i)

    # 検証データの損失計算
    model.eval() 
    with torch.no_grad():   #検証なので学習時とは違い勾配計算不要、テンソルの勾配計算を不可にしてメモリの消費を減らす
        inputs, labels = next(iter(dataloader_valid))
        outputs = model(inputs)
        loss_valid = criterion(outputs, labels)

    # ログを出力
    print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, loss_valid: {loss_valid:.4f}')  

10671
epoch: 1, loss_train: 0.4767, loss_valid: 0.3393
10671
epoch: 2, loss_train: 0.3181, loss_valid: 0.2980
10671
epoch: 3, loss_train: 0.2893, loss_valid: 0.2851
10671
epoch: 4, loss_train: 0.2738, loss_valid: 0.2786
10671
epoch: 5, loss_train: 0.2632, loss_valid: 0.2692
10671
epoch: 6, loss_train: 0.2562, loss_valid: 0.2682
10671
epoch: 7, loss_train: 0.2511, loss_valid: 0.2647
10671
epoch: 8, loss_train: 0.2463, loss_valid: 0.2650
10671
epoch: 9, loss_train: 0.2428, loss_valid: 0.2660
10671
epoch: 10, loss_train: 0.2401, loss_valid: 0.2645


## 74.正解率の計測
問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

In [48]:
def calculate_accuracy(model, loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:  #dataloaderから
            outputs = model(inputs)
            pred = torch.argmax(outputs, dim=-1)    #予測ラベル
            #print(outputs)
            total += len(inputs)    #母数
            correct += (pred == labels).sum().item()
            #print(pred)

    return correct / total

In [49]:
acc_train = calculate_accuracy(model, dataloader_train)
acc_test = calculate_accuracy(model, dataloader_test)
print(f'正解率（学習データ）：{acc_train:.3f}')
print(f'正解率（評価データ）：{acc_test:.3f}')

正解率（学習データ）：0.920
正解率（評価データ）：0.911
